In [1]:
import pandas as pd
import pyranges as pr
from Bio import SeqIO

In [ ]:
rep1 = pd.read_csv("GSE133379_A549-G4P-hg19-rep1.narrowPeak", sep="\t", header=None)
rep2 = pd.read_csv("GSE133379_A549-G4P-hg19-rep2.narrowPeak", sep="\t", header=None)

columns = ["chrom", "start", "end", "name", "score", "strand", "signal", "pval", "qval", "summit_offset"]
rep1.columns = columns
rep2.columns = columns

peaks = pd.concat([rep1, rep2], ignore_index=True).drop_duplicates()

peaks["summit"] = peaks["start"] + peaks["summit_offset"]

g4_seq = pd.read_csv("G4_seq_peaks.bed", sep="\t", header=None)
g4_seq.columns = ["chrom", "start", "end", "name", "score", "strand"]

In [ ]:
chip_pr = pr.PyRanges(peaks.rename(columns={"chrom": "Chromosome", "start": "Start", "end": "End", "summit": "Summit"}))
g4_pr = pr.PyRanges(g4_seq.rename(columns={"chrom": "Chromosome", "start": "Start", "end": "End"}))

active_g4 = chip_pr.intersect(g4_pr).df

active_g4 = active_g4.rename(columns={
    "Chromosome": "chrom",
    "Start": "start",
    "End": "end",
    "Summit": "summit"
})

if "summit_y" in active_g4.columns:
    active_g4 = active_g4.drop(columns=["summit_y"])

print(f"Found {len(active_g4)} active G4 regions in A549")

Found 168630 active G4 regions in A549


In [ ]:
atac_a549 = pd.read_csv(
    "GSM4983058_ATAC-A549_L2_Q801602.nod_peaks.narrowPeak", 
    sep="\t", header=None
)

atac_a549.columns = [
    "chrom", "start", "end", "name", "score", "strand",
    "signal", "pval", "qval", "peak_offset"
]

atac_a549 = atac_a549[["chrom", "start", "end"]]

active_g4["is_open"] = 0 

for idx, row in active_g4.iterrows():
    overlap = atac_a549[
        (atac_a549["chrom"] == row["chrom"]) &
        (atac_a549["start"] < row["end"]) &
        (atac_a549["end"] > row["start"])
    ]
    if not overlap.empty:
        active_g4.at[idx, "is_open"] = 1

print(f"Labeled chromatin accessibility for {len(active_g4)} A549 G4 regions")


✅ Labeled chromatin accessibility for 168630 A549 G4 regions


In [ ]:
from Bio import SeqIO
import numpy as np

genome = {rec.id: str(rec.seq) for rec in SeqIO.parse("hg19.fa", "fasta")}

def get_sequence(row):
    """Extract 201bp centered on summit with robust type handling"""
    try:
        chrom = str(row["chrom"])
        summit = int(float(row["summit"])) 
        seq_start = max(0, summit - 100)   
        seq_end = summit + 101
        
        chrom_seq = genome.get(chrom, genome.get("chr"+chrom))
        if not chrom_seq:
            raise ValueError(f"Chromosome {chrom} not found in genome")
            
        seq_end = min(seq_end, len(chrom_seq))
        return chrom_seq[seq_start:seq_end].upper()
    except Exception as e:
        print(f"Error processing {row['chrom']}:{row['start']}-{row['end']}: {str(e)}")
        return None

active_g4["sequence"] = active_g4.apply(get_sequence, axis=1)

active_g4 = active_g4.dropna(subset=["sequence"])
print(f"Successfully extracted {len(active_g4)} sequences")

Successfully extracted 168630 sequences


In [ ]:
import random

def generate_negative_samples(n, gc_mean, gc_std):
    negatives = []
    chromosomes = list(genome.keys())
    
    while len(negatives) < n:
        chrom = random.choice(chromosomes)
        max_start = len(genome[chrom]) - 201
        if max_start <= 0: continue
        
        start = random.randint(0, max_start)
        seq = str(genome[chrom][start:start+201]).upper()
        gc = (seq.count("G") + seq.count("C")) / 201
        
        if abs(gc - gc_mean) < gc_std:
            negatives.append({
                "chrom": chrom,
                "start": start,
                "end": start + 201,
                "sequence": seq,
                "is_open": 0, 
                "label": 0    
            })
    
    return pd.DataFrame(negatives)

gc_content = active_g4["sequence"].apply(lambda x: (x.count("G") + x.count("C")) / 201)
negatives = generate_negative_samples(len(active_g4), gc_content.mean(), gc_content.std())

negatives["is_open"] = 0
for idx, row in negatives.iterrows():
    overlap = atac_a549[
        (atac_a549["chrom"] == row["chrom"]) &
        (atac_a549["start"] < row["end"]) &
        (atac_a549["end"] > row["start"])
    ]
    if not overlap.empty:
        negatives.at[idx, "is_open"] = 1

positives = active_g4[["sequence", "is_open"]].copy()
negatives_df = negatives[["sequence", "is_open"]].copy()

positives.loc[:, "label"] = 1 
negatives_df.loc[:, "label"] = 0

training_data = pd.concat([positives, negatives_df], ignore_index=True)
training_data = training_data.sample(frac=1, random_state=42).reset_index(drop=True)

training_data.to_csv("A549_training_dataset.csv", index=False)